In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma (
    embedding_function=embedding_function,
    collection_name='split_1_300_344',
    persist_directory='./split_1_300_344'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [2]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str

graph_builder = StateGraph(AgentState)

In [3]:
# Node는 2가지가 필요
# 1. 문서를 가져오는 retrieve
# 2. 답변을 생성하는 generate

def retrieve(state: AgentState):
    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [4]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [5]:
from langchain import hub
# prompt는 genereate, rewrite, 문서 관련성 등 프롬포트가 필요하다.
# 따라서 prompt라는 이름으로는 중복 변수 이름이 될 수 있다.
generate_prompt = hub.pull("rlm/rag-prompt")

def generate(state: AgentState):
    context = state['context']
    query = state['query']
    rag_chain = generate_prompt | llm
    response = rag_chain.invoke({'question': query, 'context': context})
    return {'answer': response.content}

/root/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
from langchain import hub
from typing import Literal
doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevance(state: AgentState) -> Literal['generate', 'rewrite']:
    query = state['query']
    context = state['context']
    print(f'context = {context}')
    doc_relevance_chain = doc_relevance_prompt | llm
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})
    print(f'doc relevance response: {response}')
    if response['Score'] == 1:
        return 'relevant'
    return 'irrelevant'

/root/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
from langchain_core.prompts import PromptTemplate

dictionary = ['사용자와 관련된 표현 -> 클라이언트']
rewrite_prompt = PromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
사전: {dictionary}
질문: {{query}}
""")

# 입력된 쿼리(query)를 변형(rewrite)하여 새로운 쿼리를 생성하는 것
def rewrite(state: AgentState):
    query = state['query']
    rewrite_chain = rewrite_prompt | llm
    response = rewrite_chain.invoke({'query': query})
    return {'query': response.content}

In [12]:
from langchain import hub

hallucination_prompt = hub.pull("langchain-ai/rag-answer-hallucination")

def check_hallucination(state: AgentState):
    answer = state['answer']
    context = state['context']
    print(f'context = {context}')
    hallucination_chain = hallucination_prompt | llm
    response = hallucination_chain.invoke({'student_answer': answer, 'documents': context})
    print(f'doc relevance response: {response}')
    if response['Score'] == 1:
        return 'generate'
    return 'end'

/root/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
query = "Bean Factory와 Application Context의 차이는?"
context = retriever.invoke(query)
generate_state = {'query': query, 'context': context}
answer = generate(generate_state)

hallucination_state = {'answer': answer, 'context': context}
check_hallucination(hallucination_state)

context = [Document(id='b27eb821-f5a4-403b-a03a-bc3ae824d398', metadata={'source': './documents/split_1_300_344.txt'}, page_content='The ServletWebServerApplicationContext\nUnder the hood, Spring Boot uses a different type of ApplicationContext for embedded servlet container support. The ServletWebServerApplicationContext is a special type of WebApplicationContext that bootstraps itself by searching for a single ServletWebServerFactory bean. Usually a TomcatServletWebServerFactory, JettyServletWebServerFactory, or UndertowServletWebServerFactory has been auto-configured.\nNOTE\nYou usually do not need to be aware of these implementation classes. Most applications are auto-configured, and the appropriate ApplicationContext and ServletWebServerFactory are created on your behalf.\nIn an embedded container setup, the ServletContext is set as part of server startup which happens during application context initialization. Because of this, beans in the ApplicationContext cannot be reliably in

'end'